In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [4]:
sql = '''
select
yat.order_id, yat.add_time, yat.allot_time, yat.agent_user_name, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods
from (select distinct order_id,add_time,agent_user_name,allot_time from db_rent.ya_agent_task where overdue_type=1) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
'''
df = query(sql)
# df.drop_duplicates(subset=['order_id'], inplace=True)

In [6]:
t_date = datetime.now().strftime('%Y%m%d%H')
df.loc[:, 'date'] = df.allot_time.dt.strftime('%Y-%m-%d')
df.loc[:, '姓名'] = df.agent_user_name.apply(lambda x: str(x).split('-')[-1])
df.loc[:, '已买断'] = np.where((df.status==8)&(df.has_actual==1)&(df.relet_days==0), 1, 0)
df.loc[:, '已归还'] = np.where((df.status.isin([8, 15, 16]))&(df.has_actual==0)&(df.relet_days==0), 1, 0)
df.loc[:, '已续租'] = np.where(df.relet_days>0, 1, 0)
df.loc[:, '仍在逾期'] = np.where((df.status==5)&(df.relet_days==0), 1, 0)
df_g = df.groupby('search_time').agg({'order_id': 'count', '已买断': 'sum', '已归还': 'sum', '已续租': 'sum', '仍在逾期': 'sum'}).rename(columns={'order_id': '归还逾期数量'})
df_g = df_g.rename_axis('下单月份')
df_g
# # # df_g.to_excel('F:/x.xlsx')
# with pd.ExcelWriter(f'D:/需求/仕程需求/归还逾期_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_g.to_excel(writer, sheet_name='归还逾期')

,归还逾期数量,已买断,已归还,已续租,仍在逾期
下单月份,,,,,
2022-07,22,4,0,18,0
2022-08,100,12,9,36,43
2022-09,81,1,1,24,55
2022-10,35,7,3,14,11
2022-11,12,0,1,1,10
2022-12,40,10,0,0,30
2023-01,120,10,8,1,101
2023-02,211,20,15,0,176
2023-03,109,11,2,1,95


In [7]:

df_new = df[(df.date>='2025-05-01')&(df.date<='2025-05-31')&(df.姓名.isin(['林大鲲','吴威宇','陈锐', '张文奔', '郑瑞杰']))]
df_new = df_new.sort_values('allot_time').groupby('order_id').head(1)
df_new_g = df_new.groupby('姓名').agg({'order_id': 'count', '已买断': 'sum', '已归还': 'sum', '已续租': 'sum', '仍在逾期': 'sum'}).rename(columns={'order_id': '分配数'})
df_new_g.loc[:, '总完成'] = df_new_g.已买断+df_new_g.已归还+df_new_g.已续租
df_new_g.loc[:, '总完成率'] = (df_new_g.总完成/df_new_g.分配数).map(lambda x: format(x, '.2%'))
df_new_g
# # df_new_g = df_new_g[['分配数', '总完成', '总完成率']]
# with pd.ExcelWriter('F:/需求/仕程需求/完成情况_5月.xlsx', engine='xlsxwriter') as writer:
#     df_new_g.to_excel(writer, sheet_name='完成率')

,分配数,已买断,已归还,已续租,仍在逾期,总完成,总完成率
姓名,,,,,,,
吴威宇,52,11,0,12,29,23,44.23%
张文奔,7,0,0,3,4,3,42.86%
林大鲲,70,15,0,16,39,31,44.29%
郑瑞杰,5,0,0,3,2,3,60.00%
陈锐,71,17,0,11,43,28,39.44%


In [8]:
sql2 = '''
select
yat.order_id, yat.add_time, yat.sort, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods
from (select order_id, add_time, sort from db_rent.ya_agent_task where overdue_type=0) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
'''
df2 = query(sql2)
df2.drop_duplicates(subset=['order_id'], inplace=True)

In [9]:
df2.loc[:, 'add_date'] = pd.to_datetime(df2.add_time.dt.date)
df2_new = df2[(df2.add_date>=pd.to_datetime('2024-10-01'))&(df2.add_date<=pd.to_datetime('2025-03-31'))]
df2_new

,order_id,add_time,sort,search_time,status,has_actual,relet_days,relet_periods,add_date
21278,351928,2024-10-01 00:30:02,[17],2023-05,8,1,184.0,7.0,2024-10-01
21280,424930,2024-10-01 00:30:02,[16],2023-06,4,0,360.0,12.0,2024-10-01
21282,425376,2024-10-01 00:30:02,[16],2023-06,4,0,122.0,5.0,2024-10-01
21283,425924,2024-10-01 00:30:02,[16],2023-06,8,1,332.0,12.0,2024-10-01
21286,500879,2024-10-01 00:30:02,[15],2023-07,4,0,62.0,3.0,2024-10-01
...,...,...,...,...,...,...,...,...,...
58986,1926032,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58987,1926180,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58988,1926626,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58991,2048920,2025-03-31 00:30:04,[3],2025-01,4,0,0.0,0.0,2025-03-31
